In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
spark = SparkSession \
    .builder \
    .appName("Data Wrangling with Spark Sql.") \
    .getOrCreate()

24/05/24 12:07:07 WARN Utils: Your hostname, zahra-laptop resolves to a loopback address: 127.0.1.1; using 192.168.178.28 instead (on interface wlo1)
24/05/24 12:07:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/24 12:07:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/24 12:07:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
path= 'data/sparkify_log_small.json'
user_log = spark.read.json(path)

In [4]:
user_log.take(1)

[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046')]

In [5]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)


In [7]:
user_log.createOrReplaceTempView("user_log_table")

In [9]:
spark.sql('select * from user_log_table').show()

+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|Showaddywaddy|Logged In|  Kenneth|     M|          112|Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
|   Lily Allen|Logged In|Elizabeth|     F|            7|   Chase|195.23873| free|Shreveport-Bossie...|   PUT|NextSong|1512718541284|     5027|      

In [10]:
# user_log.describe('artist').show()

+-------+-----------------+
|summary|           artist|
+-------+-----------------+
|  count|             8347|
|   mean|            461.0|
| stddev|            300.0|
|    min|              !!!|
|    max|ÃÂlafur Arnalds|
+-------+-----------------+


In [11]:
spark.sql('''
        select *
            from user_log_table
            limit 2
            
''').show()

+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|Showaddywaddy|Logged In|  Kenneth|     M|          112|Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
|   Lily Allen|Logged In|Elizabeth|     F|            7|   Chase|195.23873| free|Shreveport-Bossie...|   PUT|NextSong|1512718541284|     5027|      

In [14]:
spark.sql(
    '''
    select count(*)
    from user_log_table
    '''
).show()

+--------+
|count(1)|
+--------+
|   10000|
+--------+


In [17]:
spark.sql('''
select artist, song
from user_log_table
where userId==1046
''').collect()

[Row(artist='Showaddywaddy', song='Christmas Tears Will Fall'),
 Row(artist='Darius Rucker', song='Be Wary Of A Woman'),
 Row(artist='Public Enemy', song='Public Enemy No.1'),
 Row(artist='Jag Panzer', song='Reign Of The Tyrants'),
 Row(artist='Boyzone', song='Father And Son'),
 Row(artist='Hollywood Undead', song='No. 5'),
 Row(artist='Jimmy Eat World', song='Seventeen'),
 Row(artist=None, song=None),
 Row(artist='Wilco', song='War on war'),
 Row(artist='Fountains Of Wayne', song='Killermont Street'),
 Row(artist='Miike Snow', song='Black & Blue'),
 Row(artist=None, song=None),
 Row(artist=None, song=None),
 Row(artist='Yeah Yeah Yeahs', song='Heads Will Roll'),
 Row(artist='Linkin Park', song='Bleed It Out [Live At Milton Keynes]'),
 Row(artist='Coldplay', song='Clocks'),
 Row(artist='Jill Scott', song='Love Rain'),
 Row(artist='Glassjaw', song="Ry Ry's Song (Album Version)"),
 Row(artist='Michael Cretu', song='The Invisible Man'),
 Row(artist='Lonnie Gordon', song='Catch You Baby (S